In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
from retrying import retry
@retry
def get_brand_links():
    url = "https://www.amazon.com/unlocked-cell-phones/b/ref=sv_cps_1?ie=UTF8&node=2407749011"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    links = soup.find_all('div', attrs={"class":"acsUxWidget"})[2].find_all('a')
    return links

In [4]:
brand_names = ['Apple',
               'Asus',
               'Samsung',
               'LG',
               'Alcatel',
               'Huawei',
               'BLU',
               'Motorola',
               'Sony',
               'ZTE']

In [5]:
brand_links = get_brand_links()
print len(brand_links)
for link in brand_links:
    print link['href']

10
https://www.amazon.com/gp/search/ref=sr_nr_p_89_3/ref=s9_acss_bw_cg_unlocked_2a1_w?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A!2335753011%2Cn%3A7072561011%2Cn%3A2407749011%2Cp_89%3AApple&bbn=2407749011&ie=UTF8&qid=1465944831&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-5&pf_rd_r=GG4DMPJ9XWER1NK4RA4E&pf_rd_t=101&pf_rd_p=8898c72a-7bc9-464f-9378-4280fb1a8e97&pf_rd_i=2407749011
https://www.amazon.com/s/ref=s9_acss_bw_cg_unlocked_2b1_w?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A!2335753011%2Cn%3A7072561011%2Cn%3A2407749011%2Cp_89%3AAsus&bbn=2407749011&ie=UTF8&qid=1465944771&rnid=2528832011&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-5&pf_rd_r=GG4DMPJ9XWER1NK4RA4E&pf_rd_t=101&pf_rd_p=8898c72a-7bc9-464f-9378-4280fb1a8e97&pf_rd_i=2407749011
https://www.amazon.com/s/ref=s9_acss_bw_cg_unlocked_2c1_w?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A!2335753011%2Cn%3A7072561011%2Cn%3A2407749011%2Cp_89%3ASamsung&bbn=2407749011&ie=UTF8&qid=1465944803&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-5&pf_rd_r

In [44]:
def get_nextpage(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    next_url = soup.find( 'a', attrs={'title':'Next Page', 'class':'pagnNext'} )
    return next_url

In [46]:
url = 'https://www.amazon.com/s/ref=sr_pg_21?fst=as%3Aoff&rh=n%3A2335752011%2Cn%3A%212335753011%2Cn%3A7072561011%2Cn%3A2407749011%2Cp_89%3AApple&page=21&bbn=2407749011&ie=UTF8&qid=1498975891'
next_url =  get_nextpage(url)
if next_url:
    print 0

In [11]:
from retrying import retry
@retry
def get_phone_link(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'lxml')
    links = soup.find('ul', {'id':'s-results-list-atf'}).find_all('a', attrs={'class':'a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal'})
    return links

In [7]:
links = get_phone_link(brand_links[0]['href'])

In [8]:
print len(links)
print links[0]['href']
print links[0]['title']

24
https://www.amazon.com/Apple-Unlocked-Smartphone-Certified-Refurbished/dp/B00YD547Q6
Apple iPhone 6 16GB Factory Unlocked GSM 4G LTE Smartphone, Space Grey (Certified Refurbished)


In [9]:
def check_length(lists):
    l = len(lists[0])
    for alist in lists:
        if len(alist) != l:
            return False
    return True

In [10]:
from retrying import retry
@retry
def get_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    reviews = soup.find_all('div', attrs={'class':'a-row review-data'})
    
    stars = soup.find('div', {'id':'cm_cr-review_list'}).find_all('i', attrs={'data-hook':'review-star-rating'})
    
    votes_parent = soup.find_all('span', attrs={'class':'cr-vote-buttons cr-vote-component'})    
    votes = []
    for item in votes_parent:
        vote = item.find('span', attrs={'data-hook':'helpful-vote-statement'})
        try:
            if 'One' in vote.text.split():
                votes.append(1)
            else:
                votes.append( int(vote.text.split()[0]) )
        except:
            votes.append(0)
    
    price = soup.find('span', {'class':'a-color-price arp-price'}).text
    if check_length([reviews, stars, votes]):
        return [tag.text for tag in reviews], \
               [int(tag.text.split('.')[0]) for tag in stars], \
               votes, \
               [float(price.strip('$'))]*len(reviews)
    raise Exception('Lengths do not match in reviews, stars and votes!')

In [11]:
from retrying import retry
@retry
def get_page_number(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    page_button = soup.find_all('li', attrs={'data-reftag':'cm_cr_arp_d_paging_btm', 'class':'page-button'})
    page_num = page_button[-1].find('a')
    return int(page_num.text)

In [12]:
testurl = "https://www.amazon.com/Apple-iPhone-Unlocked-Phone-128/product-reviews/B01M1EXQY4/ref=cm_cr_getr_d_paging_btm_1?ie=UTF8&reviewerType=avp_only_reviews&pageNumber=1"
print get_page_number( testurl )

35


In [13]:
#urlbase = "https://www.amazon.com/Apple-iPhone-Unlocked-Phone-128/product-reviews/B01M1EXQY4"
urlbase = links[0]['href'].replace('/dp/', '/product-reviews/')
#print urlbase

all_reviews = []
all_stars = []
all_votes = []
all_prices = []
for i in range(1,4):
    urladd = "/ref=cm_cr_getr_d_paging_btm_%d?ie=UTF8&reviewerType=avp_only_reviews&pageNumber=%d" %(i, i)
    #urladd = "ref=cm_cr_getr_d_paging_btm_%d?&pageNumber=%d" %(i, i)"
    reviews, stars, votes, prices = get_reviews( urlbase+urladd )
    all_reviews.extend(reviews)
    all_stars.extend(stars)
    all_votes.extend(votes)
    all_prices.extend(prices)
print len(all_reviews)
print len(all_stars)
print len(all_votes)
#print all_votes
print all_reviews[-1]
#for s in all_stars:
#    print s
print all_prices[1]


30
30
30
My wife loves her new phone. Came in perfect condition, the only thing is is that she needed a SIM card before she could activate the phone (mine didn't but we bought from two different sellers).
288.0


In [14]:
import pandas as pd
test = pd.DataFrame({'Price': all_prices,
                     'Rating':all_stars,
                     'Reviews': all_reviews,                    
                     'Review Votes':all_votes})
test.head()

,Price,Rating,Review Votes,Reviews
0,288.0,1,135,"I bought 2 iPhones with this seller, both have..."
1,288.0,1,285,I don't know what they called certified refurb...
2,288.0,1,148,I ordered the iPhone 6 64 gigabyte and looking...
3,288.0,1,181,box came empty
4,288.0,1,133,Dont waste your time! They sent me a locked ph...


In [16]:
test.to_csv('./test.csv', encoding='utf-8')

In [ ]:
url = "https://www.amazon.com/Apple-iPhone-5C-16GB-White/product-reviews/B00O8EDU6U/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=avp_only_reviews"
#reviews, stars, votes, prices = get_reviews( url )
s1 = get_reviews( url )

In [1]:
import requests
from bs4 import BeautifulSoup
from retrying import retry
import pandas as pd
import os

@retry
def get_brand_links():
    url = "https://www.amazon.com/unlocked-cell-phones/b/ref=sv_cps_1?ie=UTF8&node=2407749011"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    links = soup.find_all('div', attrs={"class":"acsUxWidget"})[2].find_all('a')
    return links

@retry
def get_phone_link(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'lxml')
    links = soup.find('ul', {'id':'s-results-list-atf'}).find_all('a', attrs={'class':'a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal'})
    return links

def check_length(lists):
    l = len(lists[0])
    for alist in lists:
        if len(alist) != l:
            return False
    return True

@retry
def get_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    reviews = soup.find_all('div', attrs={'class':'a-row review-data'})
    stars = soup.find('div', {'id':'cm_cr-review_list'}).find_all('i', attrs={'data-hook':'review-star-rating'})
    
    votes_parent = soup.find_all('span', attrs={'class':'cr-vote-buttons cr-vote-component'})    
    votes = []
    for item in votes_parent:
        vote = item.find('span', attrs={'data-hook':'helpful-vote-statement'})
        try:
            if 'One' in vote.text.split():
                votes.append(1)
            else:
                votes.append( int(vote.text.split()[0]) )
        except:
            votes.append(0)
    
    price = soup.find('span', {'class':'a-color-price arp-price'}).text
    if check_length([reviews, stars, votes]):
        return [tag.text for tag in reviews], \
               [int(tag.text.split('.')[0]) for tag in stars], \
               votes, \
               [float(price.strip('$'))]*len(reviews)
    raise Exception('Lengths do not match in reviews, stars and votes!')
    
@retry
def get_page_number(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    page_button = soup.find_all('li', attrs={'data-reftag':'cm_cr_arp_d_paging_btm', 'class':'page-button'})
    page_num = page_button[-1].find('a')
    return int(page_num.text)

def save_data(brand, product_names, 
              reviews, stars, votes, prices,
              path):
    df = pd.DataFrame({'Brand': [brand]*len(prices),
                       'Product_name': product_names,
                       'Price': prices,
                       'Rating': stars,
                       'Reviews': reviews,                    
                       'Review Votes': votes})
    df.to_csv( os.path.join(path, '%s-%s.csv' %(brand,product_names[0])),
               encoding = 'utf-8' )

In [ ]:
brand_links = get_brand_links()
brand_names = ['Apple',
               'Asus',
               'Samsung',
               'LG',
               'Alcatel',
               'Huawei',
               'BLU',
               'Motorola',
               'Sony',
               'ZTE']
brands = { a:b for a,b in zip(brand_names, brand_links) }

path = './scrape_data'

for brand in brand_names:
    print brand
    links = get_phone_link( brands[brand]['href'] )
    for link in links:
        product_name = link['title']
        print product_name
        if os.path.exists( os.path.join(path, '%s-%s.csv' %(brand,product_name)) ):
            continue
        all_reviews = []
        all_stars = []
        all_votes = []
        all_prices = []
        product_names = []
        urlbase = link['href'].replace( '/dp/', '/product-reviews/' )
        urladd = "/ref=cm_cr_getr_d_paging_btm_%d?ie=UTF8&reviewerType=avp_only_reviews&pageNumber=%d" %(1, 1)
        page_num = get_page_number(urlbase+urladd)        
        for i in range( 1, page_num ):
            "/ref=cm_cr_getr_d_paging_btm_%d?ie=UTF8&reviewerType=avp_only_reviews&pageNumber=%d" %(i, i)
            reviews, stars, votes, prices = get_reviews( urlbase+urladd )
            all_reviews.extend(reviews)
            all_stars.extend(stars)
            all_votes.extend(votes)
            all_prices.extend(prices)
            product_names.extend( [product_name]*len(reviews) )
        save_data(brand, product_names, 
                  all_reviews, all_stars, all_votes, all_prices,
                  path)

Apple
Apple iPhone 6 16GB Factory Unlocked GSM 4G LTE Smartphone, Space Grey (Certified Refurbished)
Apple iPhone 6S - 16GB GSM Unlocked - Rose (Certified Refurbished)
Apple iPhone 5S 16GB Silver GSM Unlocked (Certified Refurbished)
Apple iPhone 6 Plus 16GB Factory Unlocked GSM 4G LTE Smartphone, Space Gray (Certified Refurbished)
Apple iPhone 6 16GB Factory Unlocked GSM 4G LTE Internal Smartphone - Space Gray
Apple iPhone 5C 16GB White - Unlocked Cell Phones
